In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
semana = input("Ingrese Semana a calcular: ")

Ingrese Semana a calcular: 43


# Carga de Datos

In [3]:
mainpath = "C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Data"
print("Leyendo Archivos")
ganancias_path = "Detalle Ganancia S"+semana+".csv"
deuda_path = "Detalle Deuda S"+semana+".csv"
conexion_path = "Info Conexion.csv"
garantias_path = "Garantias Turnos S"+semana+".xlsx"
print("ok")
fp_earn = os.path.join(mainpath,ganancias_path)
fp_debt = os.path.join(mainpath,deuda_path)
fp_conexion = os.path.join(mainpath,conexion_path)
fp_garantias = os.path.join(mainpath, garantias_path)

Leyendo Archivos
ok


# Data Frames

In [4]:
df_earn = pd.read_csv(fp_earn, parse_dates = ['created_at'])
df_debt = pd.read_csv(fp_debt, parse_dates = ['created_at'])
df_conexion = pd.read_csv(fp_conexion, parse_dates = ['fecha'])
df_ldam = pd.read_excel(fp_garantias, sheet_name = "La Dehesa AM")
df_ldpm = pd.read_excel(fp_garantias, sheet_name = "La Dehesa PM")
df_autos = pd.read_excel(fp_garantias, sheet_name = "Autos")

In [5]:
ids_vina = [10440, 12061, 12062, 12064, 12065, 12066, 12068, 12081,12082,12083,12084,12085,12086,12088,12391,12096,
12098,12097,12099,12138,12140,12141,12142,12208,12145,12212,12209,12211,12244,12241,12242,12250,
12249,12243,12247,12248,12246,12245,12253,12254,12302,12319,12326,12325,12310,12460]
df_vina = df_conexion[df_conexion['skid'].isin(ids_vina)]

In [6]:
df_earn = df_earn[df_earn['storekeeper_id']!=48]
df_earn = df_earn[df_earn['storekeeper_id']!=12060]
note = df_earn.columns.tolist()
note.remove('description')
df_notificacion = pd.merge(df_earn, df_debt, how='outer', on=note)

In [60]:
df_earn[df_earn['storekeeper_id']>13000]

,storekeeper_id,created_at,name,model_id,total,description


In [8]:
df_conexion.head(10)

,skid,fecha,hora,time,orders,ganancia,tenia_cupo
0,1,2018-10-23,0,1.733333,NaN,NaN,No
1,1,2018-10-23,1,1.283333,NaN,NaN,No
2,1,2018-10-23,2,1.416667,NaN,NaN,No
3,1,2018-10-23,3,1.116667,NaN,NaN,No
4,1,2018-10-23,4,1.333333,NaN,NaN,No
5,1,2018-10-23,5,1.650000,NaN,NaN,No
6,1,2018-10-23,6,1.566667,NaN,NaN,No
7,1,2018-10-23,7,1.766667,NaN,NaN,No
8,1,2018-10-23,8,1.333333,NaN,NaN,No
9,1,2018-10-23,9,1.450000,NaN,NaN,No


df_bonus25 = df_bonus25.set_index('Inicio')
df_bonus60 = df_bonus60.set_index('Inicio')

# Garantias Dias

# Totales 

In [9]:
df_debt[df_debt['name'].str.startswith("order")]

,dispersion_id,storekeeper_id,created_at,name,model_id,total
2959,473952,12213,2018-10-25 10:15:14.431,order_not_delivered_manual_debt,5867,-27200.0
3423,476669,12710,2018-10-25 14:46:46.429,order_not_delivered_manual_debt,5886,-5200.0
5531,490447,11496,2018-10-26 16:47:42.247,order_not_delivered_manual_debt,5924,-25200.0


In [10]:
df_calcGarantias = df_conexion
df_resumen = df_conexion
df_resumen = df_resumen.drop(labels = ['hora'], axis =1)

In [11]:
dias_trabajados = df_resumen.groupby(['skid','fecha'])

In [12]:
#Totales Minutos, Ganancias, etc
df_resumen = df_resumen.groupby('skid').sum()

In [13]:
df_resumen['time']=df_resumen['time'].apply(lambda x: x/60)

In [14]:
df_calcGarantias.insert(6,"Total Pedidos",0)

In [15]:
df_calcGarantias.insert(7,"garantia horas",0)

In [16]:
df_calcGarantias.insert(8,"garantia especial",0)

In [17]:
#df con horas para garantia
df_calcGarantias = df_calcGarantias.fillna(0)

In [18]:
dias_trabajados = dias_trabajados.sum().reset_index()

In [19]:
#Suma total Dias Trabajados por RT
dtrab = dias_trabajados.groupby('skid').fecha.count()
dtrab.head(10)

skid
1        7
10042    1
10045    7
10047    7
10048    6
10059    5
10061    6
10065    5
10066    4
10067    2
Name: fecha, dtype: int64

In [20]:
df_debt = df_debt.sort_values('storekeeper_id',ascending = True)
df_debt['total'] = df_debt['total'].astype(int)

In [21]:
df_debt[df_debt['storekeeper_id']== 10042]

,dispersion_id,storekeeper_id,created_at,name,model_id,total
58,458887,10042,2018-10-23 12:06:23.132,debt_by_balance,17,-6401


In [22]:
df_earn = df_earn.sort_values('storekeeper_id',ascending = True)
df_earn['total'] = df_earn['total'].astype(int)

In [23]:
df_earn.columns

Index(['storekeeper_id', 'created_at', 'name', 'model_id', 'total',
       'description'],
      dtype='object')

In [24]:
#df_incentivos = pd.DataFrame(df_earn.groupby('description').total.sum())

In [25]:
#df_incentivos

Probar Despues:
for i in df_calcGarantias.index:
    df_calcGarantias['garantia horas'][i] = getMonto(25,df_calcGarantias['hora'][i],3)

In [26]:
earn_res = df_earn.groupby('storekeeper_id')

In [27]:
earn_res = earn_res.total.sum()

In [28]:
df_ganancia = pd.DataFrame(earn_res)

In [29]:
df_ganancia.head(10)

,total
storekeeper_id,
1,5302
10042,1400
10045,142350
10047,114825
10048,57750
10059,23125
10061,32275
10065,73975
10066,6650


In [30]:
df_debt[df_debt['storekeeper_id']==10042]

,dispersion_id,storekeeper_id,created_at,name,model_id,total
58,458887,10042,2018-10-23 12:06:23.132,debt_by_balance,17,-6401


In [31]:
debt_res = df_debt.groupby(['storekeeper_id','name'])

In [32]:
debt_res = pd.DataFrame(debt_res.total.sum())

In [33]:
debt_res.head(10)

total
storekeeper_id name                   
1              cash_base         -5302
819            pay_debt            923
10042          debt_by_balance   -6401
10043          debt_by_balance  -28000
10045          cash_base       -108601
10046          debt_by_balance  -12500
10047          cash_base        -93552
               debt_by_balance  -21204
10048          cash_base        -39450
10049          debt_by_balance -351765

In [34]:
#Resumen total deuda
df_deuda = pd.DataFrame(debt_res)
df_deuda = df_deuda.groupby('storekeeper_id').total.sum()

In [35]:
df_deuda2 = pd.DataFrame(df_deuda)

In [36]:
df_deuda2.head(10)

,total
storekeeper_id,
1,-5302
819,923
10042,-6401
10043,-28000
10045,-108601
10046,-12500
10047,-114756
10048,-39450
10049,-351765


In [37]:
df_earn[df_earn['storekeeper_id']==10059].sum()

storekeeper_id                                               201180
name              tipservice_costservice_costtipservice_costsurc...
model_id                                                    7267315
total                                                         23125
description                                                       0
dtype: object

In [38]:
#Total Ganado Semana
df_ganancia['total'].values.ravel().sum()

66751080

In [39]:
#Total Cobros Semana
df_debt['total'].values.ravel().sum()

-66355098

In [40]:
df_final = pd.merge(df_ganancia,df_deuda2,how='outer',on = 'storekeeper_id')
df_final = df_final.fillna(0).sort_values('storekeeper_id')
df_final.insert(2,'Total a Pagar', df_final.total_x + df_final.total_y)

In [41]:
df_garantias = df_earn[df_earn['name']=='service_cost']

In [42]:
#Total Cobros Semana
df_debt['total'].values.ravel().sum()

-66355098

In [43]:
#Dataframe con Total de Ordenes por RT
df_ppay = df_garantias.groupby('storekeeper_id').total.count()
df_ppay2 = pd.DataFrame(df_ppay)
df_ppay2 = df_ppay2.reset_index()

In [44]:
df_debt[df_debt['name']=='order_not_delivered_manual_debt'].sort_values('storekeeper_id')

,dispersion_id,storekeeper_id,created_at,name,model_id,total
5531,490447,11496,2018-10-26 16:47:42.247,order_not_delivered_manual_debt,5924,-25200
2959,473952,12213,2018-10-25 10:15:14.431,order_not_delivered_manual_debt,5867,-27200
3423,476669,12710,2018-10-25 14:46:46.429,order_not_delivered_manual_debt,5886,-5200


In [45]:
df_final = df_final.rename(columns= {'total_x':'Total Produccion S/Garantias','total_y':'Total Deuda'})

In [46]:
df_final = df_final.reset_index()
df_final[df_final['storekeeper_id']==10736]

,storekeeper_id,Total Produccion S/Garantias,Total Deuda,Total a Pagar


In [47]:
def createDummies(df,var_name):
    dummy = pd.get_dummies(df[var_name], prefix = var_name)
    df = df.drop(var_name, axis=1)
    df = pd.concat([df,dummy], axis =1)
    return df

In [48]:
print("Creando Archivo Resumen..")
writer = pd.ExcelWriter('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Resumen S'+semana+'.xlsx')
df_earn.to_excel(writer,'Resumen Ganancia')
df_debt.to_excel(writer,'Resumen Deuda')
df_final.to_excel(writer,'Prod-Deuda')
df_calcGarantias.to_excel(writer, 'Calculo Garantias')
df_ppay2.to_excel(writer, 'Ordenes Totales')
df_resumen.to_excel(writer, 'Totales')
writer.save()
writer.close()
print("Archivo resumen guardado en: C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Resumen S"+semana+".xlsx")

Creando Archivo Resumen..
Archivo resumen guardado en: C:/Users/mromo/Desktop/Rappi/Datasets/S43/Resumen S43.xlsx


In [49]:
dtrab.to_excel('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Dias Trabajados.xlsx')
df_final.to_excel('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Pagos S'+semana+'.xlsx')
print("Archivo de Pagos Guardado en: C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Pagos S"+semana+".xlsx")

Archivo de Pagos Guardado en: C:/Users/mromo/Desktop/Rappi/Datasets/S43/Pagos S43.xlsx


In [50]:
pagos_vina = df_earn[df_earn['storekeeper_id'].isin(ids_vina)]

In [51]:
pagos_vina.groupby('storekeeper_id').total.sum().reset_index()

,storekeeper_id,total
0,10440,153500
1,12062,83400
2,12065,130850
3,12066,302325
4,12084,354525
5,12085,45000
6,12086,377750
7,12096,341375
8,12097,15950
9,12098,222125


In [52]:
#df_incentivos.to_excel('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Detalle Incentivos.xlsx')

In [53]:
df_vina.to_excel('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Info Conexion '+semana+' - Viña.xlsx')

In [54]:
df_notificacion = pd.DataFrame(df_notificacion.groupby(['storekeeper_id','name']).total.sum()).reset_index()

In [55]:
df_notificacion

,storekeeper_id,name,total
0,1,cash_base,-5303.2
1,1,service_cost,5303.2
2,819,pay_debt,923.0
3,10042,debt_by_balance,-6401.0
4,10042,service_cost,1400.0
5,10043,debt_by_balance,-28000.0
6,10045,cash_base,-108601.0
7,10045,service_cost,117700.0
8,10045,surcharge_paid_for_rappi,7650.0
9,10045,tip,17000.0


In [56]:
df_test = createDummies(df_notificacion, 'name')

In [57]:
df_test

,storekeeper_id,total,name_additional_distance,name_cash_base,name_debt_by_balance,name_equipment_debt,name_lost_card_manual_debt,name_order_not_delivered_manual_debt,name_other_incentive,name_past_debt_manual_debt,...,name_products_reimbursement,name_referral,name_sampling,name_sampling_debt_type_expired,name_sampling_debt_type_validation,name_service_cost,name_special_incentive,name_surcharge_paid_for_rappi,name_surcharge_paid_for_user,name_tip
0,1,-5303.2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,5303.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,819,923.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10042,-6401.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10042,1400.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,10043,-28000.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,10045,-108601.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,10045,117700.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,10045,7650.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,10045,17000.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
